# AI Coursework Project

#### **Table of Contents**

-  **Section 1: Generating the Dataset**
- **Section 2: Encoder Decoder**
- **Section 3: AutoEncoder**
- **Section 4: Functions class: containing all functions used in encoder, decoder and training**
- **Section 5: Initializing The Model & Training**
- **Section 6: Plotting the results**
- **Section 7: Testing the model(Using pickle test data)**

Below you can find the code generating the data according to different random sinusoidal functions $\{f_a\}$. We randomly generate a set of 40 points in the x-axis in the interval $[-2, 2]$, slightly randomly shifted. Our functions will have the form of $y = f_a(x) = a * sin(x+a)$ where each $a$ will be randomly sampled for each function from interval $[-2, 2]$. 

## 1. Generating the Dataset

I have redesigned the provided code such that it is embedded in a Gen_Dataset class. not only will this make it work with the dataloader, but will allow us to generate a test dataset to try.

In [1]:
import torch
from itertools import chain
import random
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
import sys
import numpy as np


class Gen_Dataset(Dataset):
    """
    Generating Dataset Class

    This class uses the provided data generation functions.
    """

    def __init__(self, n_functions):
        # Sinosodial function
        Nf = n_functions
        Npts = 40
        self.x = torch.zeros(Nf, Npts, 1)
        for k in range(Nf):
            self.x[k, :, 0] = torch.linspace(-2, 2, Npts)
        self.x += torch.rand_like(self.x) * 0.1
        a = -2 + 4 * torch.rand(Nf).view(-1, 1).repeat(1, Npts).unsqueeze(2)
        self.y = a * torch.sin(self.x + a)
        self.n_samples = self.x.shape[0]

    def __getitem__(self, index):
        # Slicing Method
        return self.x[index], self.y[index]

    def __len__(self):
        # len method
        return self.n_samples

ModuleNotFoundError: No module named 'google.colab'

## 2. Encoder Decoder
Below is the code used for the encoder and decoder. they are initialized in the autoencoder such that I only use 1 model for the optimizer and during training. 

 - Output size for the encoder is rDim.
 - Input size of decoder is rDim + 1, which is the output of rDim + xt column.
 - Output size for the decoder is hard-coded to 1, which is ypred.
 - only 2 hidden layers used in this MLP Encoder Decoder Model.

In [ ]:
class Encoder(nn.Module):
    """
    Encoder

    Takes as input Xc and Yc context pairs, passes the context pairs through network,
    applying some activation function resulting in rc. Mean function applied
    to rc and rC is returned.
    """

    def __init__(self, inputSize, hiddenSize, rDim):
        super(Encoder, self).__init__()

        # Initialising parameters
        self.outputSize = rDim

        # Hidden Layers
        self.Layer1 = nn.Linear(inputSize, hiddenSize)
        self.ReLU = nn.ReLU()
        self.Layer2 = nn.Linear(hiddenSize, self.outputSize)

    def forward(self, contextPairs):
        # Layer 1 > relu > layer 2 > mean > out
        rc = self.ReLU(self.Layer1(contextPairs))
        rc = self.Layer2(rc)
        rC = torch.mean(rc, dim=1)
        return rC


class Decoder(nn.Module):
    """
    Decoder

    Takes as input rC and Xt, passes the inputs through network,
    applying some activation function resulting in y_hat. y_hat is
    returned
    """
    def __init__(self, hiddenSize, rDim):
        super(Decoder, self).__init__()

        # Initialising parameters
        self.outputSize = 1

        # Hidden Layers
        self.Layer1 = nn.Linear(rDim + 1, hiddenSize)
        self.ReLU = nn.ReLU()
        self.Layer2 = nn.Linear(hiddenSize, self.outputSize)

    def forward(self, decoderInput):
        # layer1 > relu > layer 2 > output
        decoderInput = self.ReLU(self.Layer1(decoderInput))
        ypred = self.Layer2(decoderInput)

        return ypred



## 3. AutoEncoder
The autoencoder combines the Encoder and Decoder. I initialize the AutoEncoder with the parameters required by the encoder and decoder. 

 - The if statements in the forward part of the encoder relate to how I did my test plots:
  - All inputs in the forward method of this class are initialised with some value.
  - If encoderOnly == true, context pairs are passed through xt and the values are sent to the encoder only, returning rC.
  - If decoderOnly == true, decoder inputs are passed through xt and the values are sent to the decoder only, returning ypred.
  - Formatting functions are only operated in the autoencoder, to keep the encoder and decoder as simple as possible. All methods used can be found in the Functions class found below this block.

In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self, inputSize, hiddenSize, rDim):
      super(AutoEncoder, self).__init__()

      # Initializing rDim, inputSize / hiddenSize only needed in encoder / decoder
      self.rDim = rDim

      # Initializing encoder / decoder
      self.encoder = Encoder(inputSize, hiddenSize, rDim)
      self.decoder = Decoder(hiddenSize, rDim)

    def forward(self, xt=None, yt=None, encoderOnly=False, decoderOnly=False):
      # If statements only true when using test data, otherwise works as normal
      if encoderOnly==True:
        rC = self.encoder(xt)
        return rC
      if decoderOnly==True:
        y_hat = self.decoder(xt)
        return y_hat

      # Taking in xtyt of batch and concatenating
      x = torch.cat((xt, yt), dim=2)

      # Getting context pairs and passing to the encoder
      contextPairs = Functions.get_context(self, x)
      rC = self.encoder(contextPairs)

      # Formatting xt rC and passing it to decoder
      x = Functions.format_input(self, xt, rC)
      ypred = self.decoder(x)

      # Returning prediction
      return ypred

## 4. Functions class: containing all functions used in encoder, decoder and training
 - **get context** method formats the xc and yc pairs. firstly selecting nc number of pairs, and creating a list of indices. The list is then used to select context pairs from each function. Each batch will have the same number of Nc values, but different batches will have different numbers of context pairs.
- min Nc = 5
- max Nc = 35
- randomly selected indeces

 - **format_input** method takes rc and xt to combine them into an appropriate format for the decoder

 - **Train** method contains the training loop, which outputs the loss at each epoch

In [ ]:
class Functions(AutoEncoder):
    def __init__(self):
        super(Functions, self).__init__()

    def get_context(self, batchIn):
        """
        Firstly creating an empty list, then randomly selecting a number between 5 and 35 which will be the
        number of context pairs. This is then passed into a for loop, for each selected index, select the
        context pair located at that index, and append it to the context pairs tensor.

        Tensor then split into xc and yc values.

        :param batchIn: takes the batch in, used for indexing and accessing xt / yt pairs
        :return: chosen xc, yc values as well as both formatted as pairs
        """
        global xc, yc

        # selection list, this will be the index list used to select pairs at random.
        selection = []

        self.batchSizeIn = batchIn.shape[0]

        # Randomization: min of 5 context pairs and max of 35
        n = random.randint(5, 35)
        for i in range(0, n):
            #index ranges between 0-39, so any of the 40 examples from a function can be selected
            n = random.randint(0, 39)
            selection.append(n)

        # Empty 3D tensor to hold the context pairs.
        contextPairs = torch.empty(0, len(selection), 2)

        # for each batch, then for each function, select context pairs and append to contextPairs tensor
        for batch in range(0, self.batchSizeIn):
            context = None
            for index in selection:
                c = batchIn[batch, index, :]
                if context is None:
                    context = c.unsqueeze(1)
                else:
                    context = torch.cat((context, c.unsqueeze(1)), dim=1)

            context = context.T
            contextPairs = torch.cat((contextPairs, context.unsqueeze(0)), dim=0)
        xc, yc = contextPairs[:, :, :1], contextPairs[:, :, 1:2]

        return contextPairs

    def format_input(self, xtIn, rCIn):
        """
        Formats xt and rC into an appropriate input to pass into the decoder.
        :param xtIn: xt values in, tensor will always be [batchsize, 40, 1]
        :param rCIn: rC values in, tensor will always be [batchsize, rDim]
        :return: formatted input appropriate for decoder
        """
        #Adding a dimension to rC, then combining rC and xt
        rCIn = rCIn.unsqueeze(1)
        decoderInput = torch.cat((xtIn, rCIn.expand(self.batchSizeIn, 40, self.rDim)), dim=2)

        return decoderInput

    def train(num_epochs, dataloader, criterion, optimizer, model):
        """
        Trains the model on the functions and context pairs extracted from
        the dataloader
        :param dataloader: the iterable dataloader
        :param criterion: the loss function to use
        :param optimizer: Adam or SGD
        :param model: pass in the AutoEncoder
        :return: predictions
        """
        #Training Loop
        print('Beginning to train...\n')
        global xt, yt
        for epoch in range(num_epochs):
            for (xt, yt) in dataloader:
              xt, yt = xt, yt
              y_hat = model(xt, yt)
              loss = criterion(y_hat, yt)
              optimizer.zero_grad()
              loss.backward()
              optimizer.step()

            print(f'Epoch:\t{epoch + 1}/{num_epochs}\t\tLoss:\t{loss.item():4f}')

        print('\nTraining Complete.')
        return y_hat


## 5 Initializing The Model & Training
All hyperparameters listed at the top.

 - dataloader generates 2000 functions
 - autoencoder (2 MLPS (Encoder and Decoder)) initialized
 - Loss criterion initialized (MSE)
 - Optimizer Initialized (adam or SGD)


Training loop is performed by passing in all appropriate inputs

In [ ]:
# Setting batch size
batchSize = 20
learning_rate = 0.001
weight_decay = 0.00005
hiddenSize = 30
rDim = 3
num_epochs = 75

# Generating Dataset (Training Data and Test Data)
trainData = Gen_Dataset(2000)

# Dataloader
trainLoader = DataLoader(dataset=trainData, batch_size=batchSize, shuffle=True, num_workers=2)

#Model Initialization
autoencoder = AutoEncoder(inputSize=2, hiddenSize=hiddenSize, rDim=rDim)

# Loss Criterion Used
meanSquaredError = nn.MSELoss()

# Optimizers
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=learning_rate, weight_decay=weight_decay)
#optimizer = torch.optim.SGD(autoencoder.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Training
ypred = Functions.train(num_epochs=num_epochs,
                dataloader=trainLoader,
                criterion=meanSquaredError,
                optimizer=optimizer,
                model=autoencoder)

## 6. Plotting the results

Firstly, the plot below shows how well the model performs on explicitly the last function in the batch.

In [ ]:
fig, ax = plt.subplots(figsize = (15,10))

ax.set_facecolor('whitesmoke')
plt.plot(xt[0,:,0].to('cpu'), yt[0,:,0].to('cpu'), color = 'cornflowerblue', 
         linewidth = 2, label = 'Function Pairs (xt, yt)')
plt.plot(xc[0,:,0].to('cpu'), yc[0,:,0].to('cpu'), '*', color='orange',
         label='Context Pairs (xc, yc)')
plt.plot(xt[0,:,0].to('cpu'), ypred[0,:,0].detach().to('cpu'), 
         color = 'darkseagreen', linewidth = 2, label = 'Model Predictions')
plt.suptitle('Model Evaluation (Performed on last batch)', fontsize = 25)
plt.xlabel('X Function Inputs', fontsize = 15)
plt.ylabel('Outputs From Sinosodial Functions', fontsize = 15)
plt.legend(fontsize = 12)
plt.grid(alpha = 0.2)

## 7. Testing the model (Using Pickle test data)
I have defined one method which does 3 main things:
 - extract each functions' context pairs and xt values.
 - Processes them through our model, previously trained in previous sections.
 - Plots the models predictions against the context pairs provided.

In [6]:
import pickle
path_to_the_pickle = '/content/gdrive/MyDrive/Colab Notebooks/test_data.pkl'
test_data =pickle.load(open(path_to_the_pickle,'rb'))

def get_prediction(function):
  # 6 test functions
  name= 'function_num_{}'.format(function)
  xc=test_data[name]['context_pairs'][0]
  yc=test_data[name]['context_pairs'][1]
  xt=test_data[name]['x']

  testContext = torch.cat((xc, yc), dim=2)
  rCtest = autoencoder(xt=testContext, encoderOnly=True)

  rCtest = rCtest.unsqueeze(1)
  rCtest.expand(1, xc.shape[1], -1)
  decoderInput = torch.cat((xt, rCtest.expand(1, xt.shape[1], -1)), dim=2)

  ypred = autoencoder(xt=decoderInput, decoderOnly=True)

  fig, ax = plt.subplots(figsize = (8,6))
  ax.set_facecolor('whitesmoke')
  plt.plot(xc[0,:,0].to('cpu'), yc[0,:,0].to('cpu'), '*', color='orange',
         label='Test Context Pairs (xc, yc)')
  plt.plot(xt[0,:,0].to('cpu'), ypred[0,:,0].detach().to('cpu'), 
         color = 'darkseagreen', linewidth = 2, label = 'Model Predictions')
  plt.suptitle(f'Model Evaluation Against Test Function {function}', fontsize = 20)
  plt.xlabel('X Function Inputs', fontsize = 12)
  plt.ylabel('Outputs From Sinosodial Functions', fontsize = 12)
  plt.legend(fontsize = 12)
  plt.grid(alpha = 0.2)
  

FileNotFoundError: [Errno 2] No such file or directory: '/content/gdrive/MyDrive/Colab Notebooks/test_data.pkl'

In [7]:
get_prediction(function=1)

NameError: name 'get_prediction' is not defined

In [ ]:
get_prediction(function=2)

In [2]:
get_prediction(function=3)

NameError: name 'get_prediction' is not defined

In [3]:
get_prediction(function=4)

NameError: name 'get_prediction' is not defined

In [4]:
get_prediction(function=5)

NameError: name 'get_prediction' is not defined

In [5]:
get_prediction(function=6)

NameError: name 'get_prediction' is not defined